# CURRENCIES AND BTC HISTORICAL DATA

In [1]:
%matplotlib notebook

In [46]:
# Import dependencies

import os
import csv
import pandas as pd
import json
import requests
import string
import numpy as py
from datetime import timezone


In [3]:
# Establish URL connection to coingecko

url1 = "https://api.coingecko.com/api/v3/"
print(requests.get(url1).json)

# https://api.coingecko.com/api/v3/simple/price?ids=btc&vs_currencies=eth

<bound method Response.json of <Response [404]>>


In [7]:
# Supported currencies

user_currency = str(input("What currency would you like to compare against?"))
supported_currencies = requests.get(f"{url1}simple/supported_vs_currencies").json()

if user_currency in supported_currencies:
    print(f"Data for {user_currency} is available.")
else:
    print(f"Data for {user_currency} is not available.")

What currency would you like to compare against?eur
Data for eur is available.


In [9]:
# BTC current exchange rates 

user_vs_btc = str(input("What currency would you like to compare against Bitcoin (BTC)?"))
btc_exchange_rates = requests.get(f"{url1}exchange_rates").json()
btc_exchange_rates['rates'][user_vs_btc]

What currency would you like to compare against Bitcoin (BTC)?usd


{'name': 'US Dollar', 'unit': '$', 'value': 58276.466, 'type': 'fiat'}

In [10]:
# BTC historical exchange rate (from n days back, n =1-30 or "max"); against a currency (a symbol, e.g. "usd"))
# Also... Market Caps, and Total Volumes

user_currency = str(input("What currency would you like to compare against Bitcoin (BTC)?"))
n_days = str(input("Include hourly updates from how many days back (1-30 or 'max')?"))

historical_exchange = requests.get(f"{url1}coins/bitcoin/market_chart?vs_currency={user_currency}&days={n_days}").json()

print(historical_exchange)

len(historical_exchange['prices'])

What currency would you like to compare against Bitcoin (BTC)?usd
Including how many days back (1-30 or 'max')?30
{'prices': [[1613607194415, 52354.33106806803], [1613611985108, 52228.26803271614], [1613614045259, 52183.506133621464], [1613617594403, 52323.19102271347], [1613621370337, 51905.70841194988], [1613625439029, 52163.214648583176], [1613628154945, 52145.038800077054], [1613632086763, 51909.079549463866], [1613635477933, 51992.40196948642], [1613639100452, 51904.70068004216], [1613642943565, 51820.288855346684], [1613646098361, 51665.4776656716], [1613649818860, 51391.90391143333], [1613653335817, 51531.63956470861], [1613657544475, 51886.5132956805], [1613660664863, 52141.260866132485], [1613664406645, 51586.017410455745], [1613668606868, 51950.89679140388], [1613671335866, 51864.88540055738], [1613675115642, 51914.92200440512], [1613678847350, 51857.88552848], [1613682091940, 51985.39766367797], [1613685877515, 52107.07594995904], [1613689320460, 51834.34543644542], [1613693

721

In [27]:
generic_30_day = "https://api.coingecko.com/api/v3/coins/{coin}/market_chart?vs_currency={currency}&days=30&interval=daily"

coins = ["bitcoin", "ethereum", "dogecoin"]
currencies = ["usd", "eur", "hkd"]

# Storing data for past 30 days. Most recent (03/19/2021is at the end)
# Price lists
btc_usd = []
btc_eur = []
btc_hkd = []

eth_usd = []
eth_eur = []
eth_hkd = []

doge_usd = []
doge_eur = []
doge_hkd = []

# Volume lists (USD only)
btc_vol = []
eth_vol = []
doge_vol = []

# Iterating through each coin and each currency to get 30 days of price and volume
for coin in coins:
    usd_response = requests.get(f"https://api.coingecko.com/api/v3/coins/{coin}/market_chart?vs_currency=usd&days=30&interval=daily").json()
    usd_prices = usd_response['prices']
    usd_volumes = usd_response['total_volumes']
    
    for price in usd_prices:
        if coin == 'bitcoin':
            btc_usd.append(price[1])
        elif coin == 'ethereum':
            eth_usd.append(price[1])
        elif coin == 'dogecoin':
            doge_usd.append(price[1])
            
    for volume in usd_volumes:
        if coin == 'bitcoin':
            btc_vol.append(volume[1])
        elif coin == 'ethereum':
            eth_vol.append(volume[1])
        elif coin == 'dogecoin':
            doge_vol.append(volume[1])
    
    
    eur_response = requests.get(f"https://api.coingecko.com/api/v3/coins/{coin}/market_chart?vs_currency=eur&days=30&interval=daily").json()
    eur_prices = eur_response['prices']
    
    for price in eur_prices:
        if coin == 'bitcoin':
            btc_eur.append(price[1])
        elif coin == 'ethereum':
            eth_eur.append(price[1])
        elif coin == 'dogecoin':
            doge_eur.append(price[1])
            
    hkd_response = requests.get(f"https://api.coingecko.com/api/v3/coins/{coin}/market_chart?vs_currency=hkd&days=30&interval=daily").json()
    hkd_prices = hkd_response['prices']
    
    for price in hkd_prices:
        if coin == 'bitcoin':
            btc_hkd.append(price[1])
        elif coin == 'ethereum':
            eth_hkd.append(price[1])
        elif coin == 'dogecoin':
            doge_hkd.append(price[1])

In [47]:
# Making data dictionaries to insert into pandas dataframes

days_ago = list(range(30))

btc_data = {
    'Coin': 'Bitcoin',
    'Symbol': 'BTC',
    'Days Ago': days_ago,
    'Total Vol (USD)': sorted(btc_vol, reverse=True),
    'USD': sorted(btc_usd, reverse=True),
    'EUR': sorted(btc_eur, reverse=True),
    'HKD': sorted(btc_hkd, reverse=True)
}

eth_data = {
    'Coin': 'Ethereum',
    'Symbol': 'ETH',
    'Days Ago': days_ago,
    'Total Vol (USD)': sorted(eth_vol[1::], reverse=True),
    'USD': sorted(eth_usd[1::], reverse=True),
    'EUR': sorted(eth_eur[1::], reverse=True),
    'HKD': sorted(eth_hkd[1::], reverse=True),
}

doge_data = {
    'Coin': 'Dogecoin',
    'Symbol': 'DOGE',
    'Days Ago': days_ago,
    'Total Vol (USD)': sorted(doge_vol, reverse=True),
    'USD': sorted(doge_usd, reverse=True),
    'EUR': sorted(doge_eur, reverse=True),
    'HKD': sorted(doge_hkd, reverse=True)
}

btc_df = pd.DataFrame(btc_data)
eth_df = pd.DataFrame(eth_data)
doge_df = pd.DataFrame(doge_data)

btc_df

,Coin,Symbol,Days Ago,Total Vol (USD),USD,EUR,HKD
0,Bitcoin,BTC,0,1.166896e+11,61315.197933,51295.129602,475956.158193
1,Bitcoin,BTC,1,1.015320e+11,59428.967566,49729.090338,461340.538029
2,Bitcoin,BTC,2,7.735179e+10,59014.925632,49256.512604,458264.470966
3,Bitcoin,BTC,3,7.435720e+10,58772.257254,49196.199518,456434.165671
4,Bitcoin,BTC,4,7.314059e+10,57922.409593,48605.879622,449767.510487
5,Bitcoin,BTC,5,7.139724e+10,57788.868103,48228.104398,448356.088957
6,Bitcoin,BTC,6,7.059449e+10,57669.303527,47981.150784,447117.607254
7,Bitcoin,BTC,7,6.733276e+10,57353.861472,47746.846624,445206.481983
8,Bitcoin,BTC,8,6.460523e+10,56830.253335,47559.182587,441301.124710
9,Bitcoin,BTC,9,6.356055e+10,56377.633478,46979.341207,437101.148229


In [11]:
# Historical Price for BTC

user_coin = str(input("What is the name of the coin you would like to see price data for?"))
date = str(input("For which date (DD-MM-YYYY format)?"))

historical_price = requests.get(f"{url1}coins/{user_coin}/history?date={date}").json()

historical_price

What is the name of the coin you would like to see price data for?ethereum
For which date (DD-MM-YYYY format)?16-03-2021


{'id': 'ethereum',
 'symbol': 'eth',
 'name': 'Ethereum',
 'localization': {'en': 'Ethereum',
  'de': 'Ethereum',
  'es': 'Ethereum',
  'fr': 'Ethereum',
  'it': 'Ethereum',
  'pl': 'Ethereum',
  'ro': 'Ethereum',
  'hu': 'Ethereum',
  'nl': 'Ethereum',
  'pt': 'Ethereum',
  'sv': 'Ethereum',
  'vi': 'Ethereum',
  'tr': 'Ethereum',
  'ru': 'эфириум',
  'ja': 'イーサリアム',
  'zh': '以太坊',
  'zh-tw': '以太幣',
  'ko': '이더리움',
  'ar': 'يثريوم',
  'th': 'Ethereum',
  'id': 'Ethereum'},
 'image': {'thumb': 'https://assets.coingecko.com/coins/images/279/thumb/ethereum.png?1595348880',
  'small': 'https://assets.coingecko.com/coins/images/279/small/ethereum.png?1595348880'},
 'market_data': {'current_price': {'aed': 6578.876969342337,
   'ars': 163074.54869956075,
   'aud': 2311.7018802614134,
   'bch': 3.4163974996073065,
   'bdt': 151721.26293426,
   'bhd': 675.2769805276529,
   'bmd': 1791.047851830107,
   'bnb': 7.060357868442238,
   'brl': 10058.524735877887,
   'btc': 0.03187399488579353,
   'c

# NEWS ARTICLES BY KEYWORD


In [30]:
from config import news_key
from nltk.tokenize import word_tokenize

In [13]:
# Search through the NewsAPI for articles with a specific keyword in the title

query_keyword = str(input("What keyword would you like to query?"))
query_date = str(input("For which date (YYYY-MM-DD format)?"))

query_url = f"https://newsapi.org/v2/everything?q={query_keyword}&from={query_date}&sortBy=popularity&apiKey={news_key}"

response = requests.get(query_url).json()

print(response)
print(len(response['articles']))

# https://newsapi.org/v2/everything?q={Apple}&from=2021-03-16&sortBy=popularity&apiKey=API_KEY

What keyword would you like to query?ethereum
For which date (YYYY-MM-DD format)?2021-03-16
{'status': 'ok', 'totalResults': 214, 'articles': [{'source': {'id': 'techcrunch', 'name': 'TechCrunch'}, 'author': 'Zack Whittaker', 'title': '$5.7M stolen in Roll crypto heist after hot wallet hacked', 'description': 'A security breach at cryptocurrency platform Roll allowed a hacker to obtain the private key to its hot wallet and steal its contents — worth about $5.7 million. In a statement, the company said it was investigating the breach, which happened early Sunday. “A…', 'url': 'http://techcrunch.com/2021/03/16/5-7m-stolen-in-roll-crypto-heist-after-hot-wallet-hacked/', 'urlToImage': 'https://techcrunch.com/wp-content/uploads/2021/03/cryptocurrency-coins.jpg?w=600', 'publishedAt': '2021-03-16T14:19:24Z', 'content': 'A security breach at cryptocurrency platform Roll allowed a hacker to obtain the private key to its hot wallet and steal its contents worth about $5.7 million.\r\nIn a stateme

In [24]:
# DataFrame of articles from query_date with headline containing query_keyword
articles_df = pd.DataFrame(response['articles'])
articles_df.head(5)

,source,author,title,description,url,urlToImage,publishedAt,content
0,"{'id': 'techcrunch', 'name': 'TechCrunch'}",Zack Whittaker,$5.7M stolen in Roll crypto heist after hot wa...,A security breach at cryptocurrency platform R...,http://techcrunch.com/2021/03/16/5-7m-stolen-i...,https://techcrunch.com/wp-content/uploads/2021...,2021-03-16T14:19:24Z,A security breach at cryptocurrency platform R...
1,"{'id': None, 'name': 'Gizmodo.com'}",Tom McKay,Elon's Doing Another Bit,"If you want a picture of the future of art, im...",https://gizmodo.com/elons-doing-another-bit-18...,https://i.kinja-img.com/gawker-media/image/upl...,2021-03-16T18:50:00Z,"If you want a picture of the future of art, im..."
2,"{'id': 'mashable', 'name': 'Mashable'}",Danica D'Souza,"A beginner's guide to NFTs, the crypto potenti...",Here's everything you need to know about non-f...,https://mashable.com/video/what-is-an-nft-expl...,https://mondrian.mashable.com/2021%252F03%252F...,2021-03-16T18:30:28Z,Here's everything you need to know about non-f...
3,"{'id': 'ars-technica', 'name': 'Ars Technica'}",Timothy B. Lee,Nvidia accidentally releases driver to un-nerf...,Nvidia hobbled the RTX 3060's mining abilities...,https://arstechnica.com/tech-policy/2021/03/nv...,https://cdn.arstechnica.net/wp-content/uploads...,2021-03-16T17:59:28Z,6 with 6 posters participating\r\nWhen the val...
4,"{'id': None, 'name': 'Hipertextual'}",Santiago Lago,Saca el mejor provecho de las remesas convirti...,El fenómeno migratorio que se ha desatado en V...,https://hipertextual.com/2021/03/remesas-bitco...,https://images.unsplash.com/photo-160955449679...,2021-03-16T11:11:05Z,El fenómeno migratorio que se ha desatado en V...


In [38]:
headlines = articles_df['content']
headlines_list = list(headlines)

headlines_list[0]
    

'A security breach at cryptocurrency platform Roll allowed a hacker to obtain the private key to its hot wallet and steal its contents worth about $5.7 million.\r\nIn a statement, the company said it wa… [+1908 chars]'

# SENTIMENT ANALYSIS w/ NLTK


In [18]:
import nltk
from pprint import pprint

In [15]:
nltk.download([
...     "names",
...     "stopwords",
...     "state_union",
...     "twitter_samples",
...     "movie_reviews",
...     "averaged_perceptron_tagger",
...     "vader_lexicon",
...     "punkt",
... ])

[nltk_data] Downloading package names to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package state_union to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Package state_union is already up-to-date!
[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up

True

In [20]:
words = [w for w in nltk.corpus.state_union.words() if w.isalpha()]

stopwords = nltk.corpus.stopwords.words("english")

words = [w for w in words if w.lower() not in stopwords]

words

def unique(list1):
    unique_list = []
    for x in list1:
        if x not in unique_list:
            unique_list.append(x.lower())
    return unique_list

unique_words = unique(words)

print(len(unique_words))
print(len(words))
print(unique_words)

32876
180589
['president', 'harry', 'truman', 'address', 'joint', 'session', 'congress', 'april', 'mr', 'speaker', 'mr', 'president', 'members', 'congress', 'heavy', 'heart', 'stand', 'friends', 'colleagues', 'congress', 'united', 'states', 'yesterday', 'laid', 'rest', 'mortal', 'remains', 'beloved', 'president', 'franklin', 'delano', 'roosevelt', 'time', 'like', 'words', 'inadequate', 'eloquent', 'tribute', 'would', 'reverent', 'silence', 'yet', 'decisive', 'hour', 'world', 'events', 'moving', 'rapidly', 'might', 'misunderstood', 'give', 'comfort', 'enemies', 'infinite', 'wisdom', 'almighty', 'god', 'seen', 'fit', 'take', 'us', 'great', 'man', 'loved', 'humanity', 'could', 'possibly', 'fill', 'tremendous', 'void', 'left', 'passing', 'noble', 'soul', 'ease', 'aching', 'hearts', 'untold', 'millions', 'every', 'race', 'creed', 'color', 'knows', 'lost', 'heroic', 'champion', 'justice', 'freedom', 'tragic', 'fate', 'thrust', 'upon', 'grave', 'responsibilities', 'must', 'carry', 'departed',

In [36]:


pprint(nltk.word_tokenize(text), width=79, compact=True)

TypeError: expected string or bytes-like object